In [1]:
import pandas as pd

In [2]:
path = 'C:/Users/yiyeon/TIL/school_data/'

***
## 교육여건 전처리

In [10]:
# 필요한 파일 출력

df_1 = pd.read_csv('./school_data/서울시고등학교_교육여건_합본.csv', encoding='cp949')
df_2 = pd.read_csv('./school_data/서울시고등학교_기본정보.csv', encoding='cp949')

In [11]:
# 교육여건 중 위해지표에 필요한 열만 추출
df_1.drop(df_1.columns[3:9], axis=1, inplace=True)

# 학교폭력유형_합계 열 추가
df_1['학교폭력유형건수_합계'] = df_1['신체폭력']+df_1['언어폭력']+df_1['금품갈취']+df_1['강요']+df_1['따돌림']+df_1['성폭력']+df_1['사이버폭력']+df_1['기타']

In [12]:
df_2 = df_2[['학교코드','지역','법정동코드','위도','경도']]

In [13]:
df = pd.merge(df_1,df_2, how='left', on='학교코드')
del df['구']

In [14]:
df.to_csv(path+'위해지표_교육여건.csv', index=False)

***
## 청소년 유해업소 전처리

In [3]:
# 서울 상권 데이터

df_3 = pd.read_csv(path+'소상공인시장진흥공단_상가정보_서울.csv', encoding='utf-8', sep='|')
df_3.drop(df_3.columns[19:37], axis=1, inplace=True)

In [17]:
print(df_3.info())
print(df_3.isnull().sum())

# print(df_3.nunique())
# print(df_3['표준산업분류명'].unique())
# tmp = df_3[df_3['상권업종중분류명']=='운영관리시설']
# set(tmp['상권업종소분류명'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346577 entries, 0 to 346576
Data columns (total 21 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   상가업소번호     346577 non-null  int64  
 1   상호명        346576 non-null  object 
 2   지점명        58969 non-null   object 
 3   상권업종대분류코드  346577 non-null  object 
 4   상권업종대분류명   346577 non-null  object 
 5   상권업종중분류코드  346577 non-null  object 
 6   상권업종중분류명   346577 non-null  object 
 7   상권업종소분류코드  346577 non-null  object 
 8   상권업종소분류명   346577 non-null  object 
 9   표준산업분류코드   326327 non-null  object 
 10  표준산업분류명    326327 non-null  object 
 11  시도코드       346577 non-null  int64  
 12  시도명        346577 non-null  object 
 13  시군구코드      346577 non-null  int64  
 14  시군구명       346577 non-null  object 
 15  행정동코드      346577 non-null  int64  
 16  행정동명       346442 non-null  object 
 17  법정동코드      346577 non-null  int64  
 18  법정동명       346577 non-null  object 
 19  경도         346577 non-n

### 청소년 유해업소 코드 + 서울시 상권정보

In [4]:
# 청소년 유해업소 코드 데이터

code = pd.read_csv(path+'청소년유해업소_코드.csv', encoding='cp949')
code = code[['상권업종소분류코드','청소년유해업소_구분']]

code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669 entries, 0 to 668
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   상권업종소분류코드   669 non-null    object
 1   청소년유해업소_구분  669 non-null    object
dtypes: object(2)
memory usage: 10.6+ KB


In [5]:
df = pd.merge(df_3,code, how='left', on='상권업종소분류코드')
df.drop(['지점명','표준산업분류코드','시도코드','시도명','표준산업분류명','행정동코드','행정동명'], axis=1, inplace=True)
print(df.info())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 346577 entries, 0 to 346576
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   상가업소번호      346577 non-null  int64  
 1   상호명         346576 non-null  object 
 2   상권업종대분류코드   346577 non-null  object 
 3   상권업종대분류명    346577 non-null  object 
 4   상권업종중분류코드   346577 non-null  object 
 5   상권업종중분류명    346577 non-null  object 
 6   상권업종소분류코드   346577 non-null  object 
 7   상권업종소분류명    346577 non-null  object 
 8   시군구코드       346577 non-null  int64  
 9   시군구명        346577 non-null  object 
 10  법정동코드       346577 non-null  int64  
 11  법정동명        346577 non-null  object 
 12  경도          346577 non-null  float64
 13  위도          346577 non-null  float64
 14  청소년유해업소_구분  346577 non-null  object 
dtypes: float64(2), int64(3), object(10)
memory usage: 42.3+ MB
None
상가업소번호        0
상호명           1
상권업종대분류코드     0
상권업종대분류명      0
상권업종중분류코드     0
상권업종중분류명      0
상권업종소분류코드 

In [21]:
df.to_csv(path+'청소년유해업소_구분.csv', index=False)

### 반경 500m 유해업소 개수 산출

In [6]:
school = pd.read_csv(path+'위해지표_교육여건.csv')
store = pd.read_csv(path+'청소년유해업소_구분.csv')

In [7]:
print(school.info())
print(store.info())
print(store['청소년유해업소_구분'].unique)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   학교명          320 non-null    object 
 1   학교코드         320 non-null    object 
 2   개최횟수         320 non-null    int64  
 3   심의건수         320 non-null    int64  
 4   학교장의 자체해결건수  320 non-null    int64  
 5   피해학생수        320 non-null    int64  
 6   피해학생_보호조치건수  320 non-null    int64  
 7   가해학생수        320 non-null    int64  
 8   선도·교육
조치건수   320 non-null    int64  
 9   신체폭력         320 non-null    int64  
 10  언어폭력         320 non-null    int64  
 11  금품갈취         320 non-null    int64  
 12  강요           320 non-null    int64  
 13  따돌림          320 non-null    int64  
 14  성폭력          320 non-null    int64  
 15  사이버폭력        320 non-null    int64  
 16  기타           320 non-null    int64  
 17  학교폭력유형_계     320 non-null    int64  
 18  지역           320 non-null    object 
 19  법정동코드   

In [7]:
print('청소년고용금지업소', ":", store[store.청소년유해업소_구분 == '청소년고용금지업소'].shape[0])
print('청소년출입금지업소', ":", store[store.청소년유해업소_구분 == '청소년출입금지업소'].shape[0])
print('일반업소', ":", store[store.청소년유해업소_구분 == '일반업소'].shape[0])

청소년고용금지업소 : 74290
청소년출입금지업소 : 18688
일반업소 : 253599


In [9]:
# !pip install haversine
# !pip install tqdm

In [8]:
from tqdm.notebook import tqdm
from tqdm import tqdm, tqdm_notebook

from haversine import haversine

In [9]:
restriction_em = store[store.청소년유해업소_구분 == '청소년고용금지업소']
restriction = store[store.청소년유해업소_구분 == '청소년출입금지업소']
general = store[store.청소년유해업소_구분 == '일반업소']

restriction_em.shape, restriction.shape, general.shape

((74290, 15), (18688, 15), (253599, 15))

In [10]:
def close_store(school, store, column) : 
    for i in tqdm(range(len(school))) : 
        distance = [] 
        for j in range(len(store)) : 
            schoolloc = tuple(school[['경도','위도']].iloc[i])
            storeloc = tuple(store[['경도','위도']].iloc[j])
            distance.append(haversine(schoolloc, storeloc, unit = 'm'))
            
        distance = [d for d in distance if d <= 500] 
        school[column][i] = len(distance)
        
    return school

In [ ]:
school["500미터내_청소년출입금지업소개수"], school["500미터내_청소년고용금지업소개수"] = 0, 0
school = close_store(school, restriction, "500미터내_청소년출입금지업소개수")
school = close_store(school, restriction_em, "500미터내_청소년고용금지업소개수")

school.head()

  0%|                                                                                          | 0/320 [00:00<?, ?it/s]C:\Users\yiyeon\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
 57%|██████████████████████████████████████████▉                                 | 181/320 [2:01:51<2:21:17, 60.99s/it]

In [ ]:
# 열 배치 변경

col1 = school.columns[:2].to_list()
col2 = school.columns[18:22].to_list()
col3 = school.columns[2:18].to_list()
col4 = school.columns[22:24].to_list()

new_col = col1+col2+col3+col4
school = school[new_col]

In [14]:
school.to_csv(path+'위해지표_전처리_500m.csv', index=False)

,학교명,학교코드,개최횟수,심의건수,학교장의 자체해결건수,피해학생수,피해학생_보호조치건수,가해학생수,선도·교육\n조치건수,신체폭력,...,성폭력,사이버폭력,기타,학교폭력유형_계,지역,법정동코드,위도,경도,500미터내_청소년출입금지업소개수,500미터내_청소년고용금지업소개수
0,개포고등학교,S010000373,4,0,0,0,0,0,0,0,...,0,0,0,0,강남구,1168010300,37.484959,127.059405,13,60
1,경기고등학교,S010000375,4,10,9,5,2,1,4,7,...,0,2,0,10,강남구,1168010500,37.517566,127.056074,40,192
2,경기여자고등학교,S010000376,4,0,0,0,0,0,0,0,...,0,0,0,0,강남구,1168010300,37.486887,127.065567,12,81
3,국립국악고등학교,S000003540,4,0,0,0,0,0,0,0,...,0,0,0,0,강남구,1168010300,37.476230,127.051476,13,89
4,단국대학교부속소프트웨어고등학교,S010000581,7,5,1,4,4,8,14,3,...,1,0,0,5,강남구,1168010600,37.495617,127.057767,30,198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,무학여자고등학교,S010000436,8,2,0,2,2,2,6,1,...,0,0,0,3,성동구,1120010700,37.556768,127.033599,60,244
316,서울방송고등학교,S010000585,4,2,0,6,6,2,7,0,...,1,1,0,2,성동구,1120011300,37.547773,127.011036,37,157
317,성수고등학교,S010005523,4,4,0,5,1,1,3,2,...,0,0,0,3,성동구,1120011400,37.547342,127.038253,3,25
318,성수공업고등학교,S010000592,8,7,0,7,6,11,41,17,...,4,0,0,29,성동구,1120011500,37.540540,127.051298,54,291
